Монтируем гугл диск для колаба

In [23]:

from google.colab import drive

# Монтируем Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Считываем файл

In [25]:
import pandas as pd

# Путь к файлу в Google Drive
file_path = "/content/drive/MyDrive/Colab Notebooks/train.csv"

# Читаем CSV-файл
df = pd.read_csv(file_path)


Выполнение первого пункта задачи

In [54]:
# Основная информация о датасете
print("=== Информация о датасете ===")
print(df.info())

print("\n=== Описательная статистика ===")
print(df.describe())

print("\n=== Количество пропусков ===")
print(df.isnull().sum())

=== Информация о датасете ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  FirstName    891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None

=== Описательная статистика ===
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.0000

Процент выживаемости

In [71]:
survival_rate_by_class = df.groupby('Pclass')['Survived'].mean() * 100
print("\nПроцент выживаемости по классам:")
print(survival_rate_by_class)


Процент выживаемости по классам:
Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64


Самое популярное мужское и самое популярное женское имя на корабле. Типичное имя представляет из себя "Braund, Mr. Owen Harris". Значит надо эту ячейку разделить на 3. 1 - Braund, 2 - Mr, 3 - Owen Harris. Нам нужен только последний элемент. Собственно его и сохраняем в first name. Т.е. появился новый столбец в датафрейме.

In [45]:
def extract_first_name(name):
    return name.split(', ')[1].split('. ')[1].strip()

df['FirstName'] = df['Name'].apply(extract_first_name)


Проверка нового столбца

In [ ]:
print("=== Информация об обновленном датасете ===")
print(df.info())

Ищем популярные имена. Сортируем по полу, используем value_counts

In [72]:
male_names = df[df['Sex'] == 'male']['FirstName'].value_counts()
female_names = df[df['Sex'] == 'female']['FirstName'].value_counts()

print("\n=== Самые популярные имена ===")
print("Мужское:", male_names.index[0])
print("Женское:", female_names.index[0])


=== Самые популярные имена ===
Мужское: John
Женское: Mary


Популярные имена по классам.

In [73]:
print("\n=== Популярные имена по классам ===")

# Создаем словарь для хранения результатов. Ключ словаря - номер класса
popular_names_by_class = {}

# Перебираем все классы, получаем уникальные значения классов и сортируем их по возрастанию (1,2,3).
for pclass in sorted(df['Pclass'].unique()):
    print(f"\nКласс {pclass}:")

    # Фильтруем по классу и полу для каждого класса
    males_in_class = df[(df['Pclass'] == pclass) & (df['Sex'] == 'male')]
    females_in_class = df[(df['Pclass'] == pclass) & (df['Sex'] == 'female')]

    # Считаем частоту имен
    male_name_counts = males_in_class['FirstName'].value_counts()
    female_name_counts = females_in_class['FirstName'].value_counts()

    # Берем самое популярное имя (первое в отсортированном списке)
    top_male = male_name_counts.index[0]
    top_female = female_name_counts.index[0]

    print(f"Мужское: {top_male}")
    print(f"Женское: {top_female}")

    # Сохраняем результаты в словарь
    popular_names_by_class[pclass] = {'male': top_male, 'female': top_female}


=== Популярные имена по классам ===

Класс 1:
Мужское: Victor
Женское: John Bradley (Florence Briggs Thayer)

Класс 2:
Мужское: William John
Женское: Bertha

Класс 3:
Мужское: James
Женское: Mary


Задание часть таблицы с пассажирами, возраст которых больше 44 лет

В начале было видно, что в столбце AGE есть 177 шт пропущенных значений. Надо их стереть.

In [59]:
#Удаляем строки с пропущенными значениями Age
df_with_age = df.dropna(subset=['Age'])


In [64]:
#проверка выполнение команды
print("\n=== Количество пропусков проверка AGE ===")
print(df_with_age.isnull().sum())


=== Количество пропусков проверка AGE ===
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          529
Embarked         2
FirstName        0
dtype: int64


In [65]:
# Фильтруем строки, где возраст больше 44 лет
older_than_44 = df_with_age[df_with_age['Age'] > 44]
print("\n=== Пассажиры старше 44 лет ===")
print(older_than_44)


=== Пассажиры старше 44 лет ===
     PassengerId  Survived  Pclass  \
6              7         0       1   
11            12         1       1   
15            16         1       2   
33            34         0       2   
52            53         1       1   
..           ...       ...     ...   
857          858         1       1   
862          863         1       1   
871          872         1       1   
873          874         0       3   
879          880         1       1   

                                                  Name     Sex   Age  SibSp  \
6                              McCarthy, Mr. Timothy J    male  54.0      0   
11                            Bonnell, Miss. Elizabeth  female  58.0      0   
15                    Hewlett, Mrs. (Mary D Kingcome)   female  55.0      0   
33                               Wheadon, Mr. Edward H    male  66.0      0   
52            Harper, Mrs. Henry Sleeper (Myna Haxtun)  female  49.0      1   
..                                  

Столбец уже очищен от "пустышек", так что просто фильтруем его.

In [67]:
younger_than_44_male = df_with_age[(df_with_age['Age'] < 44) & (df_with_age['Sex'] == 'male')]
print("\n=== Пассажиры младше 44 лет мужского пола ===")
print(younger_than_44_male)


=== Пассажиры младше 44 лет мужского пола ===
     PassengerId  Survived  Pclass                            Name   Sex  \
0              1         0       3         Braund, Mr. Owen Harris  male   
4              5         0       3        Allen, Mr. William Henry  male   
7              8         0       3  Palsson, Master. Gosta Leonard  male   
12            13         0       3  Saundercock, Mr. William Henry  male   
13            14         0       3     Andersson, Mr. Anders Johan  male   
..           ...       ...     ...                             ...   ...   
883          884         0       2   Banfield, Mr. Frederick James  male   
884          885         0       3          Sutehall, Mr. Henry Jr  male   
886          887         0       2           Montvila, Rev. Juozas  male   
889          890         1       1           Behr, Mr. Karl Howell  male   
890          891         0       3             Dooley, Mr. Patrick  male   

      Age  SibSp  Parch            Ticke

Считаем каюты. Тоже есть пустые значения, поэтому удаляем пустые.

In [70]:
df_cabin = df.dropna(subset=['Cabin'])

# Группируем данные по Cabin и считаем количество пассажиров в каждой каюте

cabin_counts = df_cabin.groupby('Cabin').size()
n_person_cabins = cabin_counts.value_counts().sort_index()
print("\n=== Количество n-местных кабин ===")
print(n_person_cabins)



=== Количество n-местных кабин ===
1    101
2     38
3      5
4      3
Name: count, dtype: int64
